In [1]:
import pandas as pd
import json
import csv
import cplex
import time

In [2]:
def listar(x):
    rawData = []
    start = 0
    for i in range(len(x)):
        if x[i] == ' ' or x[i] == '\n':
            rawData.append(x[start:i])
            start = i + 1
    return rawData

def print_matrix(matrix):
    for row in matrix:
        for element in row:
            print(element, end=" ")
        print()
        
def ler_grafo(u):
    arquive = open(u, 'r')
    content = arquive.read()
    rawData = []
    rawData = listar(content)

    vertices, arestas = [int(v) for v in rawData[:2]]
    edgeData = [json.loads(edge) for edge in rawData[2:]]
    print(str(vertices) + ' vertices, ' + str(arestas) + ' arestas, ')
    # print(edgeData)
    m1, m2 = [], []
    for i in range(vertices):
        m1.append([0]*vertices)
        m2.append([])
    
    for i in range(0,len(edgeData)-1, 2):
        m1[edgeData[i]][edgeData[i+1]] = 1
        m1[edgeData[i+1]][edgeData[i]] = 1

    if len(m1) != 0:
        for i in  range(len(m1)):
            for j in range(len(m1[i])):
                if m1[i][j] == 1:
                    m2[i].append(j+1)
    # print_matrix(m1)
    # print()
    # print_matrix(m2)
    return m2,vertices,arestas



In [3]:
def resolve_modelo(m2,vertices,arestas,k, timethresh):
    
    cpx = cplex.Cplex()

    y = cpx.variables.add(obj=[0] * vertices,
                             lb=[0] * vertices, ub=[1] * vertices,
                             types=['B'] * vertices,
                             names=['y_%d' % (p) for p in range(1,vertices+1)])
    h = cpx.variables.add(obj=[1] * vertices,
                            lb=[0] * vertices, ub=[1] * vertices,
                            types=['B'] * vertices,
                            names=['h_%d' % (z) for z in range(1,vertices+1)])

    #Restrição 2
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([y[i] for i in range(vertices)], [1.0]*vertices)],
        senses=['E'],
        rhs= [k])


    #Restrição 3
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]], [1.0,-1.0])
                  for i in range(vertices)],
        senses=['L'] * vertices,
        rhs=[0.0] * vertices)
    
    
    #Restrição 4
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[j-1]], [1.0,-1.0])
                  for i in range(vertices) for j in m2[i]],
        senses=['L'] * (arestas*2),
        rhs=[0.0] * (arestas*2))
    
    
    #Restrição 5
    cpx.linear_constraints.add(
        lin_expr=[cplex.SparsePair([h[i]]+[y[i]]+[y[j-1] for j in m2[i]], [1.0,-1.0]+[-1.0]*len(m2[i]))
                  for i in range(vertices)],
        senses=['G'] * vertices,
        rhs=[-len(m2[i]) for i in range(vertices)])

    
    cpx.parameters.threads.set(1)
    cpx.parameters.timelimit.set(timethresh)
    cpx.objective.set_sense(cpx.objective.sense.maximize)
    
    cpx.write('model.lp')
    start_time = time.time()
    cpx.solve()
    my_time = time.time() - start_time
    status = cpx.solution.get_status()
    # if status != 101:
    #     relaxation = [cpx.solution.MIP.get_best_objective(),
    #                     cpx.solution.MIP.get_cutoff,
    #                     # cpx.solution.MIP.get_incubent_node(),
    #                     # cpx.solution.MIP.get_mip_relative_gap(),
    #                     cpx.solution.MIP.get_num_cuts(cpx.solution.MIP.cut_type.cover),
    #                     cpx.solution.MIP.get_subproblem_status()]
    # else:
    relaxation = []

    return status, cpx.solution.get_objective_value(), cpx.solution.MIP.get_best_objective(), ','.join(relaxation), cpx.solution.get_objective_value(), my_time


In [ ]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = {
    'BA': False,
    'bipartite': False,
    'lollipop': False,
    'star': False,
    'cubic': False
}
time_threshold = 900
for log_n in range(2,15):
    n = 2**log_n
    for graphClass in ["star", "cubic", "bipartite", "BA", "lollipop"]:
        for k in range(1,4):
            k = 2**(log_n + k - 4)
            filename = f'graphs/{graphClass}/{graphClass}_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            if not deuruim[graphClass]:
                m2,v,e = ler_grafo(filename)
                status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
                if status != 101:
                    deuruim[graphClass] = True
                results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
                print(f'------------------------>{filename}_k:{k} feito.')

                index += 1
            results.to_csv(f'./results4.csv')
print("Fim.")


In [ ]:
index = 0
results = pd.DataFrame(columns=['id','graph_class', 'n', 'k', 'status', 'best_solution', 'best_bound', 'linear_relaxation',
                                 'extra_params', 'result_ticks', 'result_time'])
deuruim = 0
time_threshold = 2800
if deuruim < 4:
    for log_n in range(2,15):
        n = 2**log_n
        for k in range(0,n,n/20):
            filename = f'graphs/BA/BA_{n}.txt'
            print(f'Começando {filename}_k:{k}.------------------------>')
            m2,v,e = ler_grafo(filename)
            status, best_solution, best_bound, linear_relaxation, cplex_time, my_time = resolve_modelo(m2,v,e,int(k),time_threshold)
            if status != 101:
                deuruim += 1
            results.loc[index] = [f'{filename}_{k}', graphClass, n, k, status, best_solution, best_bound, linear_relaxation, '', cplex_time, my_time]
            print(f'------------------------>{filename}_k:{k} feito.')
            index += 1
            results.to_csv(f'./results5.csv')
print("Fim.")


In [ ]:
results

In [ ]:
results